In [1]:
from scipy import io
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    from keras import regularizers
    from keras.layers import Conv1D, Conv2D, BatchNormalization, Dropout, LeakyReLU, Flatten, Activation, Dense, MaxPooling1D, MaxPooling2D
    from keras.models import Model, Sequential
    from keras.optimizers import Adam
    import keras.backend as K

Using TensorFlow backend.


In [2]:
disp = True # boolean to indicate if to print partial results

### Processing

In [11]:
train = io.loadmat("data/training_set.mat", mdict={'trainSet':'trainSet', 'trainLabels':'trainLabels'})
val = io.loadmat("data/validation_set.mat", mdict={'valSet':'valSet', 'valLabels':'valLabels'})
test = io.loadmat("data/test_set.mat", mdict={'testSet':'testSet', 'testLabels':'testLabels'})

trainSet = np.reshape(train['trainSet'], newshape=(8370, 28, 28, 4), order=(3,0,1,2))
valSet = np.reshape(val['valSet'], newshape=(1440, 28, 28, 4), order=(3,0,1,2))
testSet = np.reshape(test['testSet'], newshape=(9, 28, 28, 4), order=(3,0,1,2))
trainLabel = train['trainLabel'].T
valLabel = val['valLabel'].T
testLabel = test['testLabel'].T

onehot_encoder = OneHotEncoder(sparse=False)
trainLabel = onehot_encoder.fit_transform(trainLabel)
valLabel = onehot_encoder.fit_transform(valLabel)
testLabel = onehot_encoder.fit_transform(testLabel)

if disp:
    print("Training set:",
          "\ntrainSet:", trainSet.shape,
          "\ntrainLabel:", trainLabel.shape,
          "\n\nValidation set:",
          "\nvalSet:", valSet.shape,
          "\nvalLabel:", valLabel.shape,
          "\n\nTest set:"
          "\ntestSet:", testSet.shape,
          "\ntestLabel:", testLabel.shape)

Training set: 
trainSet: (8370, 28, 28, 4) 
trainLabel: (8370, 3) 

Validation set: 
valSet: (1440, 28, 28, 4) 
valLabel: (1440, 3) 

Test set:
testSet: (9, 28, 28, 4) 
testLabel: (9, 3)


Here I use a keras model to create a CNN that takes as input all the 4 channels of the RGBD signal.

In [18]:
model = Sequential()

model.add(Conv2D(filters=94,
                 kernel_size=(3,3),
                 strides=3,
                 padding='same',
                 input_shape=trainSet.shape[1:]))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2,2),
                       strides=2,
                       padding='same'))    

    
model.add(Flatten())
    
model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
model.add(LeakyReLU(alpha=0.3))
    
model.add(Dropout(0.4))

model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 10, 10, 94)        3478      
_________________________________________________________________
batch_normalization_6 (Batch (None, 10, 10, 94)        376       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 10, 10, 94)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 94)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2350)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                150464    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 64)                0         
__________

In [19]:
opt = Adam(lr=0.001)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])

model.fit(x=trainSet, y=trainLabel, batch_size=30, epochs=20,
          validation_data=(valSet, valLabel))

Train on 8370 samples, validate on 1440 samples
Epoch 1/20
8370/8370 [==============================] - 3s 409us/step - loss: 1.2940 - acc: 0.7996 - val_loss: 2.0950 - val_acc: 0.3236
Epoch 2/20
8370/8370 [==============================] - 3s 318us/step - loss: 0.5859 - acc: 0.9119 - val_loss: 1.9578 - val_acc: 0.4139
Epoch 3/20
8370/8370 [==============================] - 3s 317us/step - loss: 0.3518 - acc: 0.9481 - val_loss: 1.8665 - val_acc: 0.4000
Epoch 4/20
8370/8370 [==============================] - 3s 321us/step - loss: 0.2441 - acc: 0.9685 - val_loss: 1.5815 - val_acc: 0.4125
Epoch 5/20
8370/8370 [==============================] - 3s 319us/step - loss: 0.1974 - acc: 0.9732 - val_loss: 1.7098 - val_acc: 0.3736
Epoch 6/20
8370/8370 [==============================] - 3s 320us/step - loss: 0.2102 - acc: 0.9665 - val_loss: 1.9158 - val_acc: 0.4181
Epoch 7/20
8370/8370 [==============================] - 3s 321us/step - loss: 0.1874 - acc: 0.9751 - val_loss: 1.8430 - val_acc: 0.4194


In [14]:
predLabel = model.predict_classes(testSet)

In [15]:
print(predLabel)

[2 2 2 2 2 2 2 2 2]
